In [1]:
import pennylane as qml
import numpy as np

##### SWAP Test Demo 

In [114]:

dev = qml.device('default.qubit',wires=3,shots=1000)

@qml.qnode(dev)
def circuit():
    qml.PauliY(1)
    qml.PauliY(2)
    qml.Hadamard(0)
    qml.ctrl(qml.SWAP([1,2]),control=[0])
    qml.Hadamard(0)
    return qml.probs(0)
circuit()

tensor([1., 0.], requires_grad=True)

##### Amplitude Embedding demo

In [21]:
dev_amp = qml.device('default.qubit',wires=2)

feature = np.array([5,5,6,6])
norm = np.linalg.norm(feature)

feature_norm = feature/norm

@qml.qnode(dev_amp)
def amp_circuit(feature:list):
    qml.AmplitudeEmbedding(feature,wires=range(2),normalize=True)
    return qml.state()

print('Embedding',feature,'as',feature_norm,'\nResult after embedding',amp_circuit([5,5,6,6]))


Embedding [5 5 6 6] as [0.45267873 0.45267873 0.54321448 0.54321448] 
Result after embedding [0.45267873+0.j 0.45267873+0.j 0.54321448+0.j 0.54321448+0.j]


##### Using SWAP test with Ampiltude Embedding  

In [71]:
dev_test = qml.device('default.qubit',wires=3,shots=10000)

@qml.qnode(dev_test)
def rectangle(feature:list):
    feature = np.sort(feature)
    
    qml.AmplitudeEmbedding(feature,wires=[1,2],normalize=True)
    
    qml.Hadamard(0)
    qml.ctrl(qml.SWAP([1,2]),control=[0])
    qml.Hadamard(0)
    return qml.probs(0)

In [101]:

rectangle([2,6,2,6])

tensor([0.896, 0.104], requires_grad=True)

Doesn't work Amplitude embedding encode 2**n numbers in qubits but using SWAP test we can only compare 2 Qubits. Here we have 4 numbers that is 2 qubits. We can only compare two qubits but comparing the qubits does not give the correct result  

#### Testing Basis Encondig

In [178]:
dev_basis = qml.device('default.qubit',wires=1+8,shots=1000)

@qml.qnode(dev_basis)
def base_enc(feature:list):
    qml.BasisEmbedding(features=feature[0], wires=wires_a)
    qml.BasisEmbedding(features=feature[1], wires=wires_b)
    
    qml.Hadamard(0)
    for i in range(1,5):
        qml.ctrl(qml.SWAP([i,i+4]),control=[0])
    qml.Hadamard(0)
    return qml.probs(0)

base_enc([3,2])

tensor([0.5, 0.5], requires_grad=True)

a,b,c,d - four numbers
case - 1
ab 1 
ac
ad

bc
bd

cd 1

case - 2
ab  
ac 1
ad

bc
bd 1 

cd 

if they were sorted then we only need 2 checks in any case
but if the list is not sorted we need to 6 checks in the worst case


In [240]:
dev_basis = qml.device('default.qubit',wires=1+16,shots=1000)

wires_a = range(1,5)
wires_b = range(5,9)
wires_c = range(9,13)
wires_d = range(13,17)

def swap(wires_0,wires_1):
    #Comparing a and b - or rather checking if a == b - returns |0> on the ancilla if equal and returns |+>
    for i,j in zip(wires_0,wires_1):
        qml.ctrl(qml.SWAP([i,j]),control=[0])


@qml.qnode(dev_basis)
def base_enc(feature:list):
  
    qml.BasisEmbedding(features=feature[0], wires=wires_a)
    qml.BasisEmbedding(features=feature[1], wires=wires_b)
    qml.BasisEmbedding(features=feature[2], wires=wires_c)
    qml.BasisEmbedding(features=feature[3], wires=wires_d)
    
    qml.Hadamard(0)

    swap(wires_a,wires_b)
    swap(wires_a,wires_c)
    swap(wires_a,wires_d)

    swap(wires_b,wires_c)
    swap(wires_b,wires_d)

    swap(wires_c,wires_d)
    
    qml.Hadamard(0)

    
    return qml.probs(0)


In [11]:

def is_rectangle(sides:list):
    prob = base_enc(sides)

    print(qml.drawer.draw(base_enc)(sides))

    if prob[1] > 0:
        print(prob)
        return 0
    return 1



In [14]:
is_rectangle([3,3,2,2])

 0: ──H──────────────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●────╭●───
 1: ─╭BasisEmbedding─├SWAP─│─────│─────│─────├SWAP─│─────│─────│─────├SWAP─│─────│─────│─────│────
 2: ─├BasisEmbedding─│─────├SWAP─│─────│─────│─────├SWAP─│─────│─────│─────├SWAP─│─────│─────│────
 3: ─├BasisEmbedding─│─────│─────├SWAP─│─────│─────│─────├SWAP─│─────│─────│─────├SWAP─│─────│────
 4: ─╰BasisEmbedding─│─────│─────│─────├SWAP─│─────│─────│─────├SWAP─│─────│─────│─────├SWAP─│────
 5: ─╭BasisEmbedding─╰SWAP─│─────│─────│─────│─────│─────│─────│─────│─────│─────│─────│─────├SWAP
 6: ─├BasisEmbedding───────╰SWAP─│─────│─────│─────│─────│─────│─────│─────│─────│─────│─────│────
 7: ─├BasisEmbedding─────────────╰SWAP─│─────│─────│─────│─────│─────│─────│─────│─────│─────│────
 8: ─╰BasisEmbedding───────────────────╰SWAP─│─────│─────│─────│─────│─────│─────│─────│─────│────
 9: ─╭BasisEmbedding─────────────────────────╰SWAP─│─────│─────│─────│─────│─────│─────│─────╰SWAP
10: ─├Basi

0